In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

def load_data():
    (x_train, y_train), (x_test, y_test) = cifar100.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = load_data()


169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [3]:
import random

def create_pairs(x, y, num_pairs=50000):
    pairs, labels = [], []
    num_classes = 100
    digit_indices = [np.where(y == i)[0] for i in range(num_classes)]

    for _ in range(num_pairs):
        # Positive pair (same class)
        label = random.randint(0, num_classes - 1)
        i1, i2 = random.sample(list(digit_indices[label]), 2)
        pairs.append([x[i1], x[i2]])
        labels.append(1)

        # Negative pair (different class)
        label1, label2 = random.sample(range(num_classes), 2)
        i1, i2 = random.choice(digit_indices[label1]), random.choice(digit_indices[label2])
        pairs.append([x[i1], x[i2]])
        labels.append(0)

    return np.array(pairs), np.array(labels)

pairs_train, labels_train = create_pairs(x_train, y_train, num_pairs=25000)
pairs_test, labels_test = create_pairs(x_test, y_test, num_pairs=5000)


In [5]:
def create_siamese_network(input_shape):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape, pooling='avg')

    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dense(128, activation='relu')(x)
    return models.Model(inputs, outputs)

input_shape = x_train.shape[1:]
base_model = create_siamese_network(input_shape)

input_a = layers.Input(shape=input_shape)
input_b = layers.Input(shape=input_shape)

embedding_a = base_model(input_a)
embedding_b = base_model(input_b)

distance = layers.Lambda(lambda tensors: tf.abs(tensors[0] - tensors[1]))([embedding_a, embedding_b])


x = layers.Dense(128, activation='relu')(distance)
x = layers.BatchNormalization()(x)
x = layers.Dense(1, activation='sigmoid')(x)

siamese_model = models.Model(inputs=[input_a, input_b], outputs=x)
siamese_model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_4             │ (None, 32, 32, 3)      │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ functional (Functional)   │ (None, 128)            │     24,146,176 │ input_layer_3[0][0],   │
│                           │                        │                │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lambda (Lambda)           │ (None, 128)            │              0 │ functional[0][0],      │
│                           │                        │                │ functional[1][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         16,512 │ lambda[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 128)            │            512 │ dense_2[0][0]          │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │            129 │ batch_normalization_1… │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 24,163,329 (92.18 MB)

 Trainable params: 24,109,441 (91.97 MB)

 Non-trainable params: 53,888 (210.50 KB)

In [4]:
siamese_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                      loss='binary_crossentropy',
                      metrics=['accuracy'])


x_train_a, x_train_b = pairs_train[:, 0], pairs_train[:, 1]
x_test_a, x_test_b = pairs_test[:, 0], pairs_test[:, 1]


history = siamese_model.fit(
    [x_train_a, x_train_b], labels_train,
    validation_data=([x_test_a, x_test_b], labels_test),
    epochs=20,
    batch_size=128
)


Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 48s 69ms/step - accuracy: 0.5171 - loss: 0.7617 - val_accuracy: 0.5336 - val_loss: 0.7122
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 15s 24ms/step - accuracy: 0.5559 - loss: 0.6937 - val_accuracy: 0.5435 - val_loss: 0.7153
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.5714 - loss: 0.6790 - val_accuracy: 0.5465 - val_loss: 0.7159
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.5910 - loss: 0.6672 - val_accuracy: 0.5504 - val_loss: 0.6948
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.5876 - loss: 0.6649 - val_accuracy: 0.5433 - val_loss: 0.7105
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.6006 - loss: 0.6580 - val_accuracy: 0.5623 - val_loss: 0.6888
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.6091 - loss: 0.6532 - val_accuracy: 0.5531 - val_loss: 0.7021
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 24ms/step - accuracy: 0.6129 - loss: 0.6499 - 

In [5]:
loss, accuracy = siamese_model.evaluate([x_test_a, x_test_b], labels_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 11ms/step - accuracy: 0.5735 - loss: 0.7142
Test Loss: 0.7135, Test Accuracy: 0.5733


In [6]:
from sklearn.metrics import precision_score, recall_score, accuracy_score

def evaluate_metrics(model, pairs, labels, threshold=0.5):

    predictions = model.predict([pairs[:, 0], pairs[:, 1]])
    binary_predictions = (predictions >= threshold).astype(int).flatten()

    precision = precision_score(labels, binary_predictions)
    recall = recall_score(labels, binary_predictions)
    accuracy = accuracy_score(labels, binary_predictions)

    return precision, recall, accuracy


In [7]:
precision, recall, accuracy = evaluate_metrics(siamese_model, pairs_test, labels_test, threshold=0.5)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 14s 25ms/step
Precision: 0.6189
Recall: 0.3816
Accuracy: 0.5733
